In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from lr.models.transformers.processor import clean_df
from lr.models.transformers.BertWrapper import BertWrapper
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time
import shutil
import os
import argparse

/home/felsal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/felsal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/felsal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/felsal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [42]:
basic_hyperparams = {"local_rank": -1,
             "max_seq_length": 10,
             "overwrite_cache": False,
             "num_train_epochs": 1.0,
             "per_gpu_train_batch_size": 32,
             "per_gpu_eval_batch_size": 50,
             "gradient_accumulation_steps": 1,
             "learning_rate": 5e-5,
             "weight_decay": 0.0,
             "adam_epsilon": 1e-8,
             "max_grad_norm": 1.0,
             "max_steps": -1,
             "warmup_steps": 0,
             "save_steps": 8580,
             "no_cuda": False,
             "n_gpu": 1,
             "data_set_name": folder,
             "transformation_name": None,
             "number_of_simulations": 1000,
             "rho": rho,
             "model_name_or_path": "bert",
             "output_dir": output_dir_name,
             "random_state": random_state,
             "dgp_seed": dgp_seed,
             "fp16": False,
             "fp16_opt_level": "01",
             "device": "cpu",
             "verbose": True,
             "model_type": "bert",
             "pad_on_left": False,
             "pad_token": 0,
             "n_cores": n_cores,
             'eval_sample_size': 200,
             "pad_token_segment_id": 0,
             "mask_padding_with_zero": True,
             "base_path": "data/{}/cached_".format(folder),
             "pretrained_weights": 'bert-base-uncased'}

In [44]:
# Variables
n_cores = 1
rho = 0
random_state = 123
dgp_seed = None
n_inter = 3

folder = "toy"
output_dir_name = "bert_base_snli_search"
train = pd.read_csv("data/{}/train.csv".format(folder))
train, dev_o = train_test_split(train, test_size=0.2)

print("clean train")
train = clean_df(train, n_cores=n_cores)

print("clean dev")
dev_o = clean_df(dev_o, n_cores=n_cores)

param_grid = {"max_seq_length": range(50, 210,10),
             "num_train_epochs": np.linspace(1,2.5, 10),       
             "learning_rate": np.linspace(0.00005,1, 100),
             "weight_decay": np.linspace(0,0.1, 100),
             "adam_epsilon": np.linspace(1e-8, 0.1, 100),
             "max_grad_norm": np.linspace(0.00005,1, 100)}

all_hypers = []
all_times = []
all_accs = []

for i in range(n_inter):
    hyperparams = basic_hyperparams.copy()
    for param in param_grid:
        hyperparams[param] =  np.random.choice(param_grid[param])
    model = BertWrapper(hyperparams)
    init = time.time()
    model.fit(train)
    train_time = init - time.time()
    result = model.get_results(dev_o, mode="dev")
    acc = result.indicator.mean()
    all_hypers.appens(hyperparams)
    all_times.appens(train_time)
    all_accs.appens(acc)
    
    
    






clean train
clean dev


5156